In [1]:
import os

In [2]:
%pwd

'c:\\Users\\lenovo\\Documents\\mlopsetl_project\\ETL_pipeline_mlproject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\lenovo\\Documents\\mlopsetl_project\\ETL_pipeline_mlproject'

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [39]:
from src.ETL_pipeline_mlproject.constants import *
from src.ETL_pipeline_mlproject.utils.common import read_yaml, create_directories

In [46]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH): 
          self.config= read_yaml(config_filepath)
          self.params= read_yaml(params_filepath)
          self.schema= read_yaml(schema_filepath)

          create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config= self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [47]:
import os
import urllib.request as request
from src.ETL_pipeline_mlproject import logger
import zipfile

In [48]:
## component Data ingestion
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

## Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [50]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-02 08:32:32,344: INFO: common: yaml file: config\config.yaml loaded successfully]


EnsureError: Return value of <function read_yaml at 0x0000022B1917DCF0> of type <class 'type'> does not match annotation type <class 'box.config_box.ConfigBox'>